In [2]:
import pandas as pd
import numpy as np
import pprint as ppr
import tqdm

In [ ]:
import glob

# Lectura de datos

In [4]:
files=glob.glob("*.txt")

In [5]:
data=pd.DataFrame(files,columns=["publications"])

In [6]:
data

,publications
0,pub_1.txt
1,pub_10.txt
2,pub_2.txt
3,pub_3.txt
4,pub_4.txt
5,pub_5.txt
6,pub_6.txt
7,pub_7.txt
8,pub_8.txt
9,pub_9.txt


Ejemplo de una publicación:

In [7]:
open(data.publications[1],"r",encoding="utf-8").read()

'28th November, 2020 ---\nhttps://www.trevorsbirding.com/ ---\n\nResident nesting Galahs\nI live on the edge of the rural city of Murray Bridge which is about an hour’s drive from Adelaide, South Australia. We are blessed to have a variety of parrots, cockatoos and lorikeets in the region. One of the common birds in this family is the Galah. I am sure that if I took a census of this species over a whole year, there would be very few days pass without seeing at least a handful of these lovely parrots either resting in the trees in my garden, or flying overhead. On occasions, I have even seen flocks of many dozens through to many hundreds. They are a very common species in this area.\nEasy birding\nThe photos shown in today’s post were all taken in my garden and all within a few minutes of each other. This hollow is in an old-growth mallee tree within about twenty metres of my back veranda. I have a comfortable chair located there and I enjoy sitting there reading – or just watching the 

Se puede ver cómo se adjunta la fecha y el enlace del blog de la publicación.

# Encontrar los pájaros

## Primera opción: Filtrar entidades con spacy (finalmente descartado)

In [8]:
from spacy import displacy
import en_core_web_sm
import string

Spacy no tiene etiquetas correspondientes a especies de animales. La idea de la siguiente función es tokenizar cada publicación quedándonos únicamente con las entidades que no estén en el filtro definido como *filter_*. Como veremos más adelante, esto supone perder gran parte de la información de interés, por lo que finalmente se optó por otro método (véase siguiente apartado).

In [10]:
def entities(file):

    text=open(file,"r",encoding="utf-8").read().split("---")[-1]
    #cargamos el tockenizador
    nlp = en_core_web_sm.load()

    #eliminamos números y signos de puntuación
    text=text.translate(str.maketrans("","",string.punctuation)).translate(str.maketrans("","",string.digits))

    #tockenizamos el texto
    train = nlp(text)

    #definimos un conjunto
    ent_list= set()



    filter_=["DATE","FAC","GPE","CARDINAL","TIME","QUANTITY","LOC","WORK_OF_ART","NORP","ORDINAL"]
    
    for entity in tqdm.tqdm(train.ents):
        #filtramos las entidades de interés       
        if entity.label_ not in filter_:
            #ent_list.add(entity.label_+": "+entity.text)
            ent_list.add(entity.text)
    return(ent_list)

In [11]:
ents=data.apply(np.vectorize(entities),axis=1)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<?, ?it/s]


In [13]:
for i in range(0,9):
    print("-"*20)
    print(data.publications[i])
    print(" ")
    print(ents[i])

--------------------
pub_1.txt
 
[{'the Nisqually National Wildlife Refuge on  March  This'}]
--------------------
pub_10.txt
 
[{'Adelaide South Australia', 'fed', 'noisy Galah', 'Success', 'Galahs'}]
--------------------
pub_2.txt
 
[{'Nick', 'Madeira', 'THWACK', 'Canon'}]
--------------------
pub_3.txt
 
[{'Practice', 'Canon', 'Ill'}]
--------------------
pub_4.txt
 
[{'GREAT CORMORANT LITTLE', 'SURF', 'eBird', 'GREAT', 'Robin', 'Matt Bell', 'Lighthouse', 'ENE', 'Egrets', 'American Bittern', 'Andy Griswold', 'Common', 'Kittiwake', 'Jason Rieger Phil Rusch', 'Shorebirds', 'Phil', 'Luke Tiller', 'VEERY', 'the Milford Point BW Surf Scopers'}]
--------------------
pub_5.txt
 
[{'Bispo', 'Mental', 'Balearic', 'Castro Verde', 'Sylvia', 'Those Balearic Shearwaters', 'Connecticut Audubon Society’s EcoTravel', 'American Robin', 'The Ria de Alvor', 'Mecca of Cape Cod', 'United', 'Lisbon Sagres', 'Atlanticbreeding', 'Ria de Alvor', 'PDF', 'United Airlines', 'LAX', 'Cabo de São Vicente', 'Scopo

### Extraer las especies de Dbpedia

Ahora con Sparql extraemos todas las entradas en la base de datos *dbo* que tengan la etiqueta *dbo: Bird*.

In [4]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pprint as ppr

In [7]:
import spacy


nlp = spacy.load("en_core_web_sm")

Contamos el número de resultados:

In [8]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""

PREFIX dbo:<http://dbpedia.org/ontology/>

SELECT (COUNT(*) AS ?count) WHERE {
?x a dbo:Bird .
?x rdfs:label ?label .

FILTER (lang(?label) = 'en') 
}
""")
sparql.setReturnFormat(JSON)
ret = sparql.queryAndConvert()
n_results=float(ret["results"]["bindings"][0]["count"]["value"])

Extraemos todas las entradas y sus sinónimos (si es que los tienen) y los guardamos en un diccionario junto a su enlace a la dbpedia. Nótese que al ser una consulta bastante pesada, se ha de hacer por tramos:

In [9]:
jump=3000
lim=jump
offset=lim-jump

sparql = SPARQLWrapper("http://dbpedia.org/sparql")

species={}
while lim<n_results+jump:

    print(len(species))
    
    sparql.setQuery("""

    PREFIX dbo:<http://dbpedia.org/ontology/>

    SELECT * WHERE {
    ?x a dbo:Bird .
    ?x rdfs:label ?label .
    OPTIONAL{?x dbp:synonyms ?synonyms .}

    FILTER (lang(?label) = 'en') 
    }

    ORDER BY ?label
    LIMIT """+str(lim)+"""
    OFFSET """+str(offset))
    
    sparql.setReturnFormat(JSON)
    ret = sparql.queryAndConvert()

    for r in ret["results"]["bindings"]:
        
        species[r["label"]["value"].lower()]=[r["x"]["value"]]
        
        try:
            synonyms=r["synonyms"]["value"].split("\n")

            synonyms=[synonym.replace("*","") for synonym in synonyms]
            
            for synonym in synonyms:
                
                species[synonym.lower()]=[r["x"]["value"]]
                
            
        except:
            pass
        
    lim += jump
    offset += jump
    

0
4273
12529
16286


In [72]:
species

{'1978 atlanta falcons season': ['http://dbpedia.org/resource/1978_Atlanta_Falcons_season'],
 '1979 atlanta falcons season': ['http://dbpedia.org/resource/1979_Atlanta_Falcons_season'],
 '1985–86 pittsburgh penguins season': ['http://dbpedia.org/resource/1985–86_Pittsburgh_Penguins_season'],
 '1986–87 pittsburgh penguins season': ['http://dbpedia.org/resource/1986–87_Pittsburgh_Penguins_season'],
 '1987–88 pittsburgh penguins season': ['http://dbpedia.org/resource/1987–88_Pittsburgh_Penguins_season'],
 '1991–92 pittsburgh penguins season': ['http://dbpedia.org/resource/1991–92_Pittsburgh_Penguins_season'],
 '1993–94 pittsburgh penguins season': ['http://dbpedia.org/resource/1993–94_Pittsburgh_Penguins_season'],
 '2001–02 mighty ducks of anaheim season': ['http://dbpedia.org/resource/2001–02_Mighty_Ducks_of_Anaheim_season'],
 '2002–03 mighty ducks of anaheim season': ['http://dbpedia.org/resource/2002–03_Mighty_Ducks_of_Anaheim_season'],
 '2002–03 pittsburgh penguins season': ['http://d

### Función 'birds'

La siguiente función utiliza la  función *get_close_matches* de la librería *difflib* para que para cada entidad encuentre el string más similar (si es que lo hay) dentro de la lista de especies:

In [79]:
import difflib

def birds(entities,species):

    
    entities=list(entities[0])

    results={}
    
    for ent in tqdm.tqdm(entities):

        splited_entity=ent.split(" ")
        
        # Esto es un filtro para descartar posibles entidades formadas solo por adverbios, preposiciones, etc.
        if len(splited_entity)==1:
            doc=nlp(splited_entity[0])

            token=[token for token in doc][0]

            filter_=["PROPN","NOUN"]

            if token.pos_ not in filter_:
                    continue

        
        match=difflib.get_close_matches(ent.lower(), species.keys(), cutoff=0.8)

        
        # si se encuentra un match, lo guardamos
        if len(match) !=0:
            results[ent]=[match[0],species[match[0]][0]]
            
            
    return(results)

In [80]:
results=pd.DataFrame(ents).apply(np.vectorize(birds),species=species,axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 13.76it/s]


In [81]:
for i in range(0,len(results)):
    print("-"*100)
    print(data.publications[i])
    print(" ")
    pprint.pprint(results[i][0])
    print(" ")

----------------------------------------------------------------------------------------------------
pub_1.txt
 
{'Common Teal': ['common tern', 'http://dbpedia.org/resource/Common_tern']}
 
----------------------------------------------------------------------------------------------------
pub_10.txt
 
{}
 
----------------------------------------------------------------------------------------------------
pub_2.txt
 
{}
 
----------------------------------------------------------------------------------------------------
pub_3.txt
 
{}
 
----------------------------------------------------------------------------------------------------
pub_4.txt
 
{'American Bittern': ['american bittern',
                      'http://dbpedia.org/resource/American_bittern'],
 'Egrets': ['egret', 'http://dbpedia.org/resource/Egret'],
 'eBird': ['seabird', 'http://dbpedia.org/resource/Seabird']}
 
----------------------------------------------------------------------------------------------------
pub_

Podemos ver que , aunque sea un proceso bastante eficiente y que consigue detectar algunas especies, hay publicaciones en las que no es capaz de extraer nada, detecta poco o  directamente saca resultados erróneos.

El problema está en el paso inicial: el filtrado de entidades elimina demasiada información relevante, por lo que hay que pensar un método en el que no sea necesario filtrar parte del texto.

## Segunda opción: Utilizar el texto en crudo 

La función *get_close_matches* tiene como inputs una palabra de interés y una lista de strings de donde queremos extraer la palabra más similar, lo cual nos obliga a tokenizar el texto. Esto supone un reto en el caso en el que el string de interés está formado por dos o más palabras. 

Se ha llegado a la siguiente solución:

In [71]:
def birds(file,species):
    
    # diccionario donde guardaremos los resultados 
    matches={}
    
    text=open(file,"r",encoding="utf-8").read().split("---")[-1]
    
    #limpiamos el texto de signos de puntuación, números, mayúsculas y saltos de página
    text_tokens=text.translate(str.maketrans("","",string.punctuation)).translate(str.maketrans("","",string.digits)).lower().split()

   
    # iniciamos un bucle que recorrerá todas las especies de pájaro encontradas en la Dbpedia
    
    for sp in tqdm.tqdm(species.keys()):
        
        # definimos el parámetro beg (de beggining) cuya utilidad veremos más adelante
        beg=0
        
        # limpiamos el nombre de la especie de pájaro como se ha hecho con el texto
        sp_trans=sp.translate(str.maketrans("","",string.punctuation)).translate(str.maketrans("","",string.digits)).strip().split("(")[0]
        
        # miramos por cuántas palabras está formado el string de la especie de pájaro
        sp_len=len(sp_trans.split())
        
        # en el caso de que sea una única palabra, bucamos el string más similar dentro de la lista de tokens de la publicación
        if sp_len==1:
            
            # Nota: el parámetro 'cutoff' establece un límite en la similitud que han de tener el input y el match. Imponemos un límite bastante alto.
            
            match=difflib.get_close_matches(sp_trans,text_tokens,cutoff=0.87)
            
            #si se encuentra algún match, se guarda en el diccionario : el match como llave y la especie de pájaro junto a su enlace de la dbpedia como valor
            
            if len(match) !=0:
                    matches[match[0]]=[sp, species[sp][0]]
        
        # caso en el que esté formado por dos o más palabras. Nota: Suponemos que el nombre del pájaro estará compuesto como máximo por 4.
        if sp_len<=4 and sp_len>1:
            
            #iniciamos un bucle infinito
            while True:
                
                
                # dependiendo del número de palabras, agrupamos los strings del texto tokenizado de 2 en 2, 3 en 3 o 4 en 4
                
                #aquí entra en juego el parámetro 'beg': debemos contemplar todas las posibilidades. Por ejemplo, en el caso de que 
                # estemos buscando la especie 'red cormorant' y la frase que lo menciona dentro del texto es 'I saw a red cormorant in the countryside', 
                # la primera agrupación que se hará será ["I saw","a red", "cormorant in", ...], lo cual nos impide hacer el match que buscamos. 
                # El parámeto 'beg' desplaza una posición la agrupación, por lo que en la siguiente iteración tendremos ["saw a", "red cormorant", ...], 
                # y ya podremos hacer el match sin problemas.
                
                
                
                if sp_len==2:
                    new_text=[i+" "+j for i,j in zip(text_tokens[beg::2], text_tokens[beg+1::2])]
                if sp_len==3:
                    new_text=[i+" "+j+" "+k for i,j,k in zip(text_tokens[beg::3], text_tokens[beg+1::3],text_tokens[beg+2::3])]
                if sp_len==4:
                    new_text=[i+" "+j+" "+k+" "+z for i,j,k,z in zip(text_tokens[beg::4], text_tokens[beg+1::4],text_tokens[beg+2::4],text_tokens[beg+3::4])]

                
                # Se busca el match con el nuevo texto con los strings agrupados:
                
                match=difflib.get_close_matches(sp_trans,new_text,cutoff=0.87)
                
                # En el caso de que este se encuentre, lo guardamos y detenemos el loop
                if len(match) !=0:
                    matches[match[0]]=[sp, species[sp][0]]
                    break
                    
                # si no, se mira si 'beg' ha llegado a su límite (número de palabras de sp; precisamente el número de combinaciones posibles). Si es así, se detiene el bucle,
                # en caso contrario se suma uno a 'beg'.
                
                else:
                    if beg==sp_len:
                        break
                    else:
                        beg +=1
                    
               
                
    return(matches)
            
            
    

Aplicamos la función:

In [72]:
results=data.apply(np.vectorize(birds), species=species, axis=1)

100%|███████████████████████████████████████████████████████████████████████████| 16286/16286 [00:16<00:00, 995.52it/s]


In [80]:
results.to_pickle("results.pkl")

In [15]:
results=pd.read_pickle("results.pkl")

In [16]:
for i in range(0,len(results)):
    print(" ")
    ppr.pprint(results.iloc[i][0])
    print(" ")
    print("-"*40)

 
{'greenwinged teal': ['green-winged teal',
                      'http://dbpedia.org/resource/Green-winged_teal']}
 
----------------------------------------
 
{'cockatoos': ['cockatoo', 'http://dbpedia.org/resource/Cockatoo'],
 'parrots': ['parrot', 'http://dbpedia.org/resource/Parrot'],
 'species': ['species:', 'http://dbpedia.org/resource/Ludiortyx']}
 
----------------------------------------
 
{'gull': ['gull', 'http://dbpedia.org/resource/Gull'],
 'hope': ['(hope, 2002)', 'http://dbpedia.org/resource/Lamarqueavis'],
 'peregrine falcon': ['peregrine falcon',
                      'http://dbpedia.org/resource/Peregrine_falcon'],
 'ringbilled gull': ['ring-billed gull',
                     'http://dbpedia.org/resource/Ring-billed_gull']}
 
----------------------------------------
 
{'brant': ['branta', 'http://dbpedia.org/resource/Branta'],
 'gull': ['gull', 'http://dbpedia.org/resource/Gull'],
 'iceland gull': ['iceland gull', 'http://dbpedia.org/resource/Iceland_gull']}
 
-----

Es un proceso computacionalmente costoso, pero podemos ver que da buenos resultados. Si nos fijamos, la inmensa mayoría de ellos son correctos salvo algunas excepciones que deberían ser eliminadas posteriormente.

# Ontología

En este apartado organizaremos los resultados en una ontología que posteriormente podrá ser abierta en *Protegé*. 

In [17]:
from owlready2 import *

onto = get_ontology("http://test.org/trabajo_pajaros.owl")

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


La estructura será muy básica: 

- Cada pájaro será una individual asociado por la propiedad *IS_MENTIONED_BY* con su correspondiente publicación y por *DBPEDIA_URL* con su enlace de la Dbpedia.
- Cada publicación tendrá asociados todos los pájaros que menciona mediante la propiedad *MENTIONS*, el enlace del blog de donde se extrajo mediante *PUBLICATION_URL* y su fecha a partir de *PUBLICATION_DATE*.

In [18]:
with onto:
    class Bird(Thing):
             pass
    class Publication(Thing):
            pass
    class Dbpedia(Thing):
        pass
    
    class Pub_url(Thing):
        pass
    
    class IS_MENTIONED_BY(ObjectProperty):
            domain    = [Bird]
            range     = [Publication]
            
    class MENTIONS(ObjectProperty):
            domain    = [Publication]
            range     = [Bird]
    class DBPEDIA_URL(ObjectProperty):
            domain    = [Dbpedia]
            range     = [Bird]
            
    class PUBLICATION_URL(ObjectProperty):
        domain    = [Pub_url]
        range     = [Publication]
            
    class PUBLICATION_DATE(DataProperty):
        range     = [str]
        

Todo este bucle es para guardar los resultados dentro de la ontología:

In [19]:
for ind,result in enumerate(results):
    
    text=open(data.publications[ind],"r",encoding="utf-8").read().split("---")
    
    date=text[0].translate(str.maketrans("","","\n")).translate(str.maketrans(" ","_","_"))
    url=text[1].translate(str.maketrans("","","\n"))
    
    pub = Publication(data.publications[ind])
    url=Pub_url(url)
    
    
    pub.PUBLICATION_URL.append(url)
    pub.PUBLICATION_DATE=[date]
    
    for key in result[0].keys():
        
        my_bird = Bird(result[0][key][0].translate(str.maketrans(" ","_","_")))
        
        bird_url=Dbpedia(result[0][key][1])        
        bird_url.label=result[0][key][1]
        
        my_bird.IS_MENTIONED_BY.append(pub)
        
        my_bird.DBPEDIA_URL.append(bird_url)
        
        pub.MENTIONS.append(my_bird)
       
       

Guardamos la ontología:

In [20]:
onto.save(file="birds_ontology.xml", format = "rdfxml")